In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv("covid_early_stage_symptoms.csv")
df

,gender,age_year,fever,cough,runny_nose,muscle_soreness,pneumonia,diarrhea,lung_infection,travel_history,isolation_treatment,test_results
0,male,89,1,1,0,0,0,0,0,1,0,0
1,male,68,1,0,0,0,0,0,0,0,0,0
2,male,68,0,0,0,0,0,0,0,1,0,0
3,male,68,1,1,0,0,0,0,0,1,1,1
4,male,50,1,1,1,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
6507,female,44,1,1,0,0,0,0,0,1,0,1
6508,female,44,1,1,0,0,0,0,0,0,0,0
6509,female,58,0,0,0,0,0,0,0,0,0,0
6510,female,58,1,1,0,0,0,0,0,0,0,0


In [3]:
df.columns[df.isna().any()]

Index([], dtype='object')

In [4]:
df.describe()

,age_year,fever,cough,runny_nose,muscle_soreness,pneumonia,diarrhea,lung_infection,travel_history,isolation_treatment,test_results
count,6512.000000,6512.000000,6512.000000,6512.000000,6512.000000,6512.000000,6512.000000,6512.000000,6512.000000,6512.000000,6512.000000
mean,44.019502,0.410780,0.303286,0.084306,0.003993,0.074785,0.005682,0.131296,0.650952,0.216984,0.241400
std,16.112865,0.492013,0.459713,0.277867,0.063066,0.263064,0.075169,0.337750,0.476706,0.412223,0.427965
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,32.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,43.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,55.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,96.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
df.shape

(6512, 12)

In [6]:
df.isna().sum()

gender                 0
age_year               0
fever                  0
cough                  0
runny_nose             0
muscle_soreness        0
pneumonia              0
diarrhea               0
lung_infection         0
travel_history         0
isolation_treatment    0
test_results           0
dtype: int64

In [7]:
df.cough.value_counts()

0    4537
1    1975
Name: cough, dtype: int64

In [8]:
df.test_results.value_counts()

0    4940
1    1572
Name: test_results, dtype: int64

In [20]:
final = df.drop(['lung_infection','isolation_treatment', 'pneumonia'],axis = 'columns')

In [21]:
final

,gender,age_year,fever,cough,runny_nose,muscle_soreness,diarrhea,travel_history,test_results
0,male,89,1,1,0,0,0,1,0
1,male,68,1,0,0,0,0,0,0
2,male,68,0,0,0,0,0,1,0
3,male,68,1,1,0,0,0,1,1
4,male,50,1,1,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...
6507,female,44,1,1,0,0,0,1,1
6508,female,44,1,1,0,0,0,0,0
6509,female,58,0,0,0,0,0,0,0
6510,female,58,1,1,0,0,0,0,0


In [22]:
dummies = pd.get_dummies(final.gender)
dummies

,female,male
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
6507,1,0
6508,1,0
6509,1,0
6510,1,0


In [23]:
merged = pd.concat([final, dummies], axis = 'columns')
merged

,gender,age_year,fever,cough,runny_nose,muscle_soreness,diarrhea,travel_history,test_results,female,male
0,male,89,1,1,0,0,0,1,0,0,1
1,male,68,1,0,0,0,0,0,0,0,1
2,male,68,0,0,0,0,0,1,0,0,1
3,male,68,1,1,0,0,0,1,1,0,1
4,male,50,1,1,1,0,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
6507,female,44,1,1,0,0,0,1,1,1,0
6508,female,44,1,1,0,0,0,0,0,1,0
6509,female,58,0,0,0,0,0,0,0,1,0
6510,female,58,1,1,0,0,0,0,0,1,0


In [24]:
final_1 = merged.drop(['gender'], axis = 'columns')
final_1

,age_year,fever,cough,runny_nose,muscle_soreness,diarrhea,travel_history,test_results,female,male
0,89,1,1,0,0,0,1,0,0,1
1,68,1,0,0,0,0,0,0,0,1
2,68,0,0,0,0,0,1,0,0,1
3,68,1,1,0,0,0,1,1,0,1
4,50,1,1,1,0,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...
6507,44,1,1,0,0,0,1,1,1,0
6508,44,1,1,0,0,0,0,0,1,0
6509,58,0,0,0,0,0,0,0,1,0
6510,58,1,1,0,0,0,0,0,1,0


In [25]:
final_1 = final_1.drop(['female'], axis = 'columns')
final_1

,age_year,fever,cough,runny_nose,muscle_soreness,diarrhea,travel_history,test_results,male
0,89,1,1,0,0,0,1,0,1
1,68,1,0,0,0,0,0,0,1
2,68,0,0,0,0,0,1,0,1
3,68,1,1,0,0,0,1,1,1
4,50,1,1,1,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...
6507,44,1,1,0,0,0,1,1,0
6508,44,1,1,0,0,0,0,0,0
6509,58,0,0,0,0,0,0,0,0
6510,58,1,1,0,0,0,0,0,0


In [26]:
final_1.rename(columns = {'male':'gender'}, inplace = True)
final_1

,age_year,fever,cough,runny_nose,muscle_soreness,diarrhea,travel_history,test_results,gender
0,89,1,1,0,0,0,1,0,1
1,68,1,0,0,0,0,0,0,1
2,68,0,0,0,0,0,1,0,1
3,68,1,1,0,0,0,1,1,1
4,50,1,1,1,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...
6507,44,1,1,0,0,0,1,1,0
6508,44,1,1,0,0,0,0,0,0
6509,58,0,0,0,0,0,0,0,0
6510,58,1,1,0,0,0,0,0,0


In [28]:
X = final_1.drop(['test_results'], axis = 'columns')
X

,age_year,fever,cough,runny_nose,muscle_soreness,diarrhea,travel_history,gender
0,89,1,1,0,0,0,1,1
1,68,1,0,0,0,0,0,1
2,68,0,0,0,0,0,1,1
3,68,1,1,0,0,0,1,1
4,50,1,1,1,0,0,1,1
...,...,...,...,...,...,...,...,...
6507,44,1,1,0,0,0,1,0
6508,44,1,1,0,0,0,0,0
6509,58,0,0,0,0,0,0,0
6510,58,1,1,0,0,0,0,0


In [29]:
y = final_1.test_results
y

0       0
1       0
2       0
3       1
4       1
       ..
6507    1
6508    0
6509    0
6510    0
6511    0
Name: test_results, Length: 6512, dtype: int64

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [47]:
model = RandomForestClassifier(n_estimators=500)
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=500)

In [48]:
model.score(X_test, y_test)

0.827021494370522